# Winning Jeopardy
Jeopardy is a popular TV show in the US where participants answer questions to win money. It's been running for a few decades, and is a major force in popular culture. 
Let's say I want to compete on Jeopardy, and i am looking for any edge i can get to win. In this project, i'll work with a dataset of Jeopardy questions to figure out some patterns in the questions that could help me win.

The dataset is named jeopardy.csv, and contains 20000 rows from the beginning of a full dataset of Jeopardy questions, which you can download [here](https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file). Here's the beginning of the file:




In [32]:
from IPython.display import Image
Image(filename='C:\Users\Jeevan Reddy\Pictures\Screenshots\Screenshot(3)')


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-32-a88ad1f43a54>, line 2)

In [ ]:
import pandas as pd
jeopardy=pd.read_csv("jeopardy.csv")
#print(jeopardy.head())
print(jeopardy.columns)

In [ ]:
def clean_col(col):
    col=col.replace(" ","")
    col=col.strip()
    col=col.lower()
    return col
jeopardy.columns=[clean_col(c) for c in jeopardy.columns]

In [ ]:
print(jeopardy.columns)

In [ ]:
print(jeopardy["question"].head())
    

In [ ]:
import re
def normalize_string(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:        
        if char not in punctuations:
            no_punct = no_punct + char
    no_punct=no_punct.lower()
    #string=re.sub('''!()-[]{};:'"\,<>./?@#$%^&*_~'''," ",string)
    #string=string.translate(None,string.punctuation)
    return no_punct
jeopardy["clean_question"]=jeopardy["question"].apply(normalize_string)
jeopardy["clean_answer"]=jeopardy["answer"].apply(normalize_string)
print(jeopardy["question"].head())
print(jeopardy["answer"].head())


In [ ]:
print(jeopardy["value"].head())

In [ ]:
import numpy
def clean_value(value):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in value:        
        if char not in punctuations:
            no_punct = no_punct + char
    #no_punct=
    return no_punct
jeopardy["clean_value"]=jeopardy["value"].apply(clean_value)
jeopardy["clean_value"]=pd.to_numeric(jeopardy["clean_value"],errors='coerce').fillna(0) 

In [ ]:
jeopardy["clean_value"].value_counts()

In [ ]:
print(jeopardy.dtypes)

In [ ]:
print(jeopardy["airdate"].unique())

In [ ]:
jeopardy["airdate"]=pd.to_datetime(jeopardy["airdate"])

In [ ]:
print(jeopardy["airdate"].head())

In [ ]:
print(jeopardy.dtypes)

In [ ]:
def count_matches(row):
    split_answer = row["clean_answer"].split(" ")
    split_question = row["clean_question"].split(" ")
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

jeopardy["answer_in_question"] = jeopardy.apply(count_matches, axis=1)


In [40]:
print(jeopardy['answer_in_question'].head(20))

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.333333
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.500000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
Name: answer_in_question, dtype: float64


In [34]:
jeopardy["answer_in_question"].mean()

0.06035277385469894

# Answer terms in the question
The answer only appears in the question about 6% of the time. This isn't a huge number, and means that we probably can't just hope that hearing a question will enable us to figure out the answer. We'll probably have to study

In [35]:
question_overlap=[]
terms_used=set()
for i, row in jeopardy.iterrows():
    split_question=row["clean_question"].split(" ")
    split_question=[q for q in split_question if len(q)>5]
    match_count=0
    for word in split_question:
        if word in terms_used:
            match_count+=1
    for word in split_question:
            terms_used.add(word)
    if len(split_question) > 0:
            match_count /= len(split_question)
    question_overlap.append(match_count)
jeopardy["question_overlap"]=question_overlap
print(jeopardy["question_overlap"].head(10))
            
    


0    0.000
1    0.000
2    0.000
3    0.000
4    0.000
5    0.000
6    0.000
7    0.000
8    0.125
9    0.000
Name: question_overlap, dtype: float64


In [36]:
jeopardy["question_overlap"].mean()

0.6902117143393507

#     Question overlap
There is about 70% overlap between terms in new questions and terms in old questions. This only looks at a small set of questions, and it doesn't look at phrases, it looks at single terms. This makes it relatively insignificant, but it does mean that it's worth looking more into the recycling of questions.
    

In [41]:
def determine_value(row):
    value = 0
    if row["clean_value"] > 800:
        value = 1
    return value

jeopardy["high_value"] = jeopardy.apply(determine_value, axis=1)
print(jeopardy["high_value"].head())

0    0
1    0
2    0
3    0
4    0
Name: high_value, dtype: int64


In [47]:
five_rows=jeopardy["clean_question"].head()
print(five_rows.str.split(" "))

0    [for, the, last, 8, years, of, his, life, gali...
1    [no, 2, 1912, olympian, football, star, at, ca...
2    [the, city, of, yuma, in, this, state, has, a,...
3    [in, 1963, live, on, the, art, linkletter, sho...
4    [signer, of, the, dec, of, indep, framer, of, ...
Name: clean_question, dtype: object


In [42]:
def count_usage(term):
    low_count = 0
    high_count = 0
    for i, row in jeopardy.iterrows():
        if term in row["clean_question"].split(" "):
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

comparison_terms = list(terms_used)[:5]
print(comparison_terms)
observed_expected = []
for term in comparison_terms:
    observed_expected.append(count_usage(term))

observed_expected
        

['montana', '“chocks”', 'branford', 'participate', '6milewide']


[(6, 3), (0, 1), (1, 0), (0, 1), (1, 0)]

In [39]:
from scipy.stats import chisquare
import numpy as np

high_value_count = jeopardy[jeopardy["high_value"] == 1].shape[0]
low_value_count = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []
for obs in observed_expected:
    total = sum(obs)
    total_prop = total / jeopardy.shape[0]
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = np.array([obs[0], obs[1]])
    expected = np.array([high_value_exp, low_value_exp])
    chi_squared.append(chisquare(observed, expected))

chi_squared

[Power_divergenceResult(statistic=6.353131314909584, pvalue=0.011717429728771048),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047)]

# Chi-squared results
None of the terms had a significant difference in usage between high value and low value rows. Additionally, the frequencies were all lower than 5, so the chi-squared test isn't as valid. It would be better to run this test with only terms that have higher frequencies.